In [ ]:
import scanpy as sc
import scanpy.external as sce
import scanpy
import squidpy as sq
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import anndata
from pathlib import Path
import os 
sc.logging.print_header()
print(f"squidpy=={sq.__version__}")
output = '/home/carsten/alvira_bioinformatics/uterus/data/pilot/240724_spatial_test'
os.makedirs(output, exist_ok=True)
sc.set_figure_params(dpi=300, dpi_save=300,format="png")
sc.settings.figdir = output


In [ ]:
adata_sc = sc.read('/home/carsten/alvira_bioinformatics/uterus/data/single_cell_files/scanpy_files/uterus_processed_celltyped.gz.h5ad')

In [ ]:
gene_df = pd.read_csv('/home/carsten/alvira_bioinformatics/uterus/data/pilot/240325_merscope_planning/gene_list_trim.csv',index_col=0)
gene_df_ct_markers = gene_df.loc[gene_df['Reason'].str.contains('ct_marker')].index.to_list()

In [ ]:
ct_marker_dict = {}
for ct in adata_sc.obs['Cell Subtype'].cat.categories:
    ct_marker_dict[ct] = gene_df.loc[gene_df['Reason'].str.contains(ct)].index.to_list()
lin_marker_dict = {}
for ct in adata_sc.obs['Lineage'].cat.categories:
    lin_marker_dict[ct] = gene_df.loc[gene_df['Reason'].str.contains(ct)].index.to_list()

In [ ]:
sc.pl.DotPlot(adata_sc,['BRD4'], groupby='Cell Subtype').add_totals().style(cmap='Reds').legend(width=2).savefig(f'{output}/dotplot_BRD4.png')

In [ ]:
sc.pl.DotPlot(adata_sc,['BRD4'], groupby=['Cell Subtype','Group']).add_totals().style(cmap='Reds').legend(width=2).savefig(f'{output}/dotplot_BRD4.png')

In [ ]:
adata_spatial=sc.read('/home/carsten/alvira_bioinformatics/uterus/data/vizgen/initial_analysis/all_runs.gz.h5ad')


In [ ]:
adata_spatial

In [ ]:
sc.pl.umap(adata_spatial,color='Sample',save='sample.png')
sc.pl.umap(adata_spatial,color='Cell Subtype_ind',title='Predicted Cell Type',save='celltype.png')

In [ ]:
for sample in adata_spatial.obs['Sample'].cat.categories:
    adata_sample = adata_spatial[adata_spatial.obs['Sample']==sample]
    sq.pl.spatial_scatter(
        adata_sample,
        shape=None,
        title=sample,
        color=[
            "Cell Subtype_ind",
        ],
        wspace=0.4,
        figsize=(8,8),
        save=f'{output}/{sample}/spatial_{sample}.png'
    )
    sq.pl.spatial_scatter(
        adata_sample,
        shape=None,
        title=sample,
        color=[
            "Cell Subtype_ind",
        ],
        groups=['Capillary','Macrovascular','Vascular smooth muscle'],
        wspace=0.4,
        figsize=(8,8),
        save=f'{output}/{sample}/spatial_{sample}_vessels.png'
    )
    sq.pl.spatial_scatter(
    adata_sample,
    shape=None,
    color=[
        "VWF",
    ],
    groups=['Capillary','Macrovascular','Vascular smooth muscle'],
    wspace=0.4,
    figsize=(8,8),
    save=f'{output}/{sample}/spatial_{sample}_VWF.png'
)
    sq.pl.spatial_scatter(
        adata_sample,
        shape=None,
        color=[
            "NOTCH3",
        ],
        groups=['Capillary','Macrovascular','Vascular smooth muscle'],
        wspace=0.4,
        figsize=(8,8),
        save=f'{output}/{sample}/spatial_{sample}_NOTCH3.png'
    )

In [ ]:
sq.gr.spatial_neighbors(adata_spatial)
sq.gr.nhood_enrichment(adata_spatial,cluster_key='Cell Subtype_ind')
sq.pl.nhood_enrichment(adata_spatial,cluster_key='Cell Subtype_ind',mode='zscore',annotate=True,vmin=0,save =f'{output}/neighborhood.png')

# sq.gr.co_occurrence(adata_spatial,cluster_key='Cell Subtype_ind')
# sq.pl.co_occurrence(adata_spatial,cluster_key='Cell Subtype_ind')

In [ ]:
df = pd.DataFrame(data= adata_spatial.uns['Cell Subtype_ind_nhood_enrichment']['zscore'], index= adata_spatial.obs['Cell Subtype_ind'].cat.categories,columns=adata_spatial.obs['Cell Subtype_ind'].cat.categories)
sns.set_style('white')
fig = sns.clustermap(df,cmap='vlag',vmin=-50,vmax=50)
plt.tight_layout()
fig.savefig(f'{output}/neighborhood.png', dpi=300)

In [ ]:
sq.pl.nhood_enrichment(adata_spatial,cluster_key='Cell Subtype_ind',mode='zscore',annotate=False,method='single',save =f'{output}/neighborhood.png')


In [ ]:
sq.pl.nhood_enrichment(adata_spatial,cluster_key='Cell Subtype_ind',mode='zscore',cmap='vlag',annotate=True,)


In [ ]:
gene_df.loc['NOTUM']

### https://squidpy.readthedocs.io/en/stable/notebooks/tutorials/tutorial_vizgen.html

In [ ]:
for sample, dt in sample_dictionary.items():
    spatial_data_path = dt['path']
    os.makedirs(f'{output}/{sample}', exist_ok=True)
    sc.settings.figdir = f'{output}/{sample}'
    adata = sq.read.vizgen(path=spatial_data_path,counts_file='cell_by_gene.csv',meta_file='cell_metadata.csv')
    sc.pp.calculate_qc_metrics(adata, percent_top=(1,2,5,10,20,50,100,),inplace=True)
    adata.uns['unassinged_transcripts']=adata.obsm["blank_genes"].to_numpy().sum() / adata.var["total_counts"].sum() * 100
    sc.pp.filter_cells(adata, min_counts=10)
    sc.pp.filter_cells(adata,min_genes=2)
    sc.pp.filter_genes(adata, min_counts=1)
    sc.pl.highest_expr_genes(adata,save=True,show=False)
    adata.layers["counts"] = adata.X.copy()
    sc.pp.normalize_total(adata, inplace=True)
    sc.pp.log1p(adata,base=10)
    sc.pp.pca(adata)
    sc.pp.neighbors(adata)
    sc.tl.umap(adata)
    sc.tl.leiden(adata)
    sc.pl.umap(adata, color =['leiden'], save='leiden.png',show=False)
    sc.tl.rank_genes_groups(adata,groupby='leiden')
    sc.pl.rank_genes_groups_dotplot(adata,save='leiden_markers.png',show=False)
    sc.pl.umap(adata,color=adata.var.sort_values('total_counts',ascending=False).head(20).index,save='top20_genes_detected.png',show=False)
    sc.pl.umap(adata,color=['n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_1_genes','pct_counts_in_top_2_genes','pct_counts_in_top_5_genes', 'pct_counts_in_top_10_genes','pct_counts_in_top_50_genes','pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes'], save='counts_qc.png',show=False)
    sc.pl.violin(adata,['pct_counts_in_top_1_genes','pct_counts_in_top_2_genes','pct_counts_in_top_5_genes', 'pct_counts_in_top_10_genes','pct_counts_in_top_50_genes','pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes'],multi_panel=True, save='counts_qc.png',show=False)

    adata.layers["log1p"] = adata.X.copy()
    ct_df = pd.DataFrame(index=adata.obs_names, columns=[f'{ct} score' for ct in ct_marker_dict.keys()])
    for ct,genes in ct_marker_dict.items():
        name = f'{ct} score'
        sc.tl.score_genes(adata,genes,score_name=name)
        ct_df[name] = adata.obs[name].copy()
    adata.obs['Cell Subtype_ind'] = ct_df.idxmax(axis=1)
    sc.pl.umap(adata,color='Cell Subtype_ind',save='Cell Subtype_ind.png',show=False)
    sc.pl.umap(adata,color=[f'{ct} score' for ct in ct_marker_dict.keys()],save='celltype_marker_score.png',show=False)
    for ct,genes in lin_marker_dict.items():
        name = f'{ct} score'
        sc.tl.score_genes(adata,genes,score_name=name)
    sc.pl.umap(adata,color=[f'{ct} score' for ct in lin_marker_dict.keys()],save='lineage_marker_score.png',show=False)
    os.makedirs(f'{output}/{sample}/gene_umaps', exist_ok=True)
    sc.settings.figdir = f'{output}/{sample}/gene_umaps'
    for gene in adata.var_names:
        sc.pl.umap(adata,color=gene,save=gene,show=False)
    dt['adata'] = adata


In [ ]:
os.makedirs(f'{output}/combined', exist_ok=True)
sc.settings.figdir = f'{output}/combined'
adatas=[]
for sample, dt in sample_dictionary.items():
    spatial_data_path = dt['path']
    adata = sq.read.vizgen(path=spatial_data_path,counts_file='cell_by_gene.csv',meta_file='cell_metadata.csv')
    adata.obs['Sample'] = sample
    adatas.append(adata)
adata = anndata.concat(adatas)
sc.pp.calculate_qc_metrics(adata, percent_top=(1,2,5,10,20,50,100,),inplace=True)
adata.uns['unassinged_transcripts']=adata.obsm["blank_genes"].to_numpy().sum() / adata.var["total_counts"].sum() * 100
sc.pp.filter_cells(adata, min_counts=10)
sc.pp.filter_cells(adata,min_genes=2)
sc.pp.filter_genes(adata, min_counts=1)
sc.pl.highest_expr_genes(adata,save=True,show=False)
adata.layers["counts"] = adata.X.copy()
sc.pp.normalize_total(adata,inplace=True)
sc.pp.log1p(adata) #Normalize total leaves it at 18 counts per cell, already a small space
sce.pp.magic(adata,knn=10,random_state=7)
sc.pp.pca(adata)
sc.pl.pca_variance_ratio(adata,save=True)
sc.pp.neighbors(adata)
sc.tl.umap(adata)
sc.tl.leiden(adata,resolution=1)
sc.pl.umap(adata, color =['Sample','leiden'], save='leiden.png',show=False)
sc.tl.rank_genes_groups(adata,groupby='leiden')
sc.pl.rank_genes_groups_dotplot(adata,save='leiden_markers.png',show=False)
sc.pl.umap(adata,color=adata.var.sort_values('total_counts',ascending=False).head(20).index,save='top20_genes_detected.png',show=False)
sc.pl.umap(adata,color=['n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_1_genes','pct_counts_in_top_2_genes','pct_counts_in_top_5_genes', 'pct_counts_in_top_10_genes','pct_counts_in_top_50_genes','pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes'], save='counts_qc.png',show=False)
sc.pl.violin(adata,['pct_counts_in_top_1_genes','pct_counts_in_top_2_genes','pct_counts_in_top_5_genes', 'pct_counts_in_top_10_genes','pct_counts_in_top_50_genes','pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes'],multi_panel=True, save='pct_counts_qc.png',show=False)
sc.pl.violin(adata,['n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts',],multi_panel=True, save='counts_qc.png',show=False)

adata.layers["log1p"] = adata.X.copy()
ct_df = pd.DataFrame(index=adata.obs_names, columns=[f'{ct}' for ct in ct_marker_dict.keys()])
for ct,genes in ct_marker_dict.items():
    name = f'{ct} score'
    sc.tl.score_genes(adata,genes,score_name=name)
    ct_df[ct] = adata.obs[name].copy()
adata.obs['Cell Subtype_ind'] = ct_df.idxmax(axis=1)
sc.pl.umap(adata,color='Cell Subtype_ind',save='Cell Subtype_ind.png',show=False)

sc.pl.umap(adata,color=[f'{ct} score' for ct in ct_marker_dict.keys()],save='celltype_marker_score.png',show=False)
for ct,genes in lin_marker_dict.items():
        name = f'{ct} score'
        sc.tl.score_genes(adata,genes,score_name=name)
sc.pl.umap(adata,color=[f'{ct} score' for ct in lin_marker_dict.keys()],save='lineage_marker_score.png',show=False)
os.makedirs(f'{output}/combined/gene_umaps', exist_ok=True)
sc.settings.figdir = f'{output}/combined/gene_umaps'
for gene in adata.var_names:
    sc.pl.umap(adata,color=gene,save=gene,show=False)

In [ ]:
for ct in ct_marker_dict.keys():
    sc.pl.umap(adata,color='Cell Subtype_ind',groups=[ct])


In [ ]:
adata = sample_dictionary['Sample_01']['adata'].copy()
sample_1_leiden_dictionary = {
    '0': 'Uterine smooth muscle',
    '1':'Uterine smooth muscle',
    '2':'Uterine smooth muscle',
    '3':'Matrix Fibroblast',
    '4':'Vascular smooth muscle',
    '5':'Uterine smooth muscle',
    '6':'Macrovascular',
    '7':'Uterine smooth muscle',
    '8':'Myeloid',
    '9':'Uterine smooth muscle',
    '10':'Uterine smooth muscle',
    '11':'Uterine smooth muscle',
    '12':'Myeloid',
    '13':'Uterine smooth muscle',}
adata.obs['Cell Subtype']= [sample_1_leiden_dictionary[x] for x in adata.obs['leiden']]
sc.pl.umap(adata, color='Cell Subtype')

sc.pl.dotplot(adata,['NOTCH3','ADGRL3','DPP6','KCNMA1','VWF','PDGFRB'], groupby='Cell Subtype')

sq.pl.spatial_scatter(
    adata,
    shape=None,
    color=[
        "Cell Subtype",
    ],
    groups=['Macrovascular','Vascular smooth muscle'],
    wspace=0.4,
    figsize=(8,8)
)

sq.gr.spatial_neighbors(adata)
sq.gr.nhood_enrichment(adata,cluster_key='Cell Subtype')
sq.pl.nhood_enrichment(adata,cluster_key='Cell Subtype',mode='zscore',annotate=True,)

sq.gr.co_occurrence(adata,cluster_key='Cell Subtype')
sq.pl.co_occurrence(adata,cluster_key='Cell Subtype')

In [ ]:
adata2 = sample_dictionary['Sample_02']['adata'].copy()
sample_2_leiden_dictionary = {
    '0': 'Uterine smooth muscle',
    '1':'Uterine smooth muscle',
    '2':'Matrix Fibroblast',
    '3':'Matrix Fibroblast',
    '4':'Macrovascular',
    '5':'Myeloid',
    '6':'Uterine smooth muscle',
    '7':'Uterine smooth muscle',
    '8':'Uterine smooth muscle',
    '9':'Lymphatic',
    '10':'Endometiral',
    '11':'Myeloid_2',
    '12':'Uterine smooth muscle',
    '13':'Uterine smooth muscle',}
adata2.obs['Cell Subtype']= [sample_2_leiden_dictionary[x] for x in adata2.obs['leiden']]
sc.pl.umap(adata2, color='Cell Subtype')

sc.pl.dotplot(adata2,['NOTCH3','ADGRL3','DPP6','KCNMA1','VWF','PDGFRB'], groupby='Cell Subtype')

sq.pl.spatial_scatter(
    adata2,
    shape=None,
    color=[
        "Cell Subtype",
    ],
    groups=['Macrovascular','Vascular smooth muscle'],
    wspace=0.4,
    figsize=(8,8)
)

sq.gr.spatial_neighbors(adata2)
sq.gr.nhood_enrichment(adata2,cluster_key='Cell Subtype')
sq.pl.nhood_enrichment(adata2,cluster_key='Cell Subtype',mode='zscore',annotate=True,)

sq.gr.co_occurrence(adata2,cluster_key='Cell Subtype')
sq.pl.co_occurrence(adata2,cluster_key='Cell Subtype')

In [ ]:
sns.histplot(adata.var['log1p_total_counts'])

In [ ]:
pd.set_option('display.max_rows', 500)

adata.obs.groupby('leiden')['Cell Subtype_ind'].value_counts(normalize=True)


In [ ]:
sq.pl.spatial_scatter(
    adata[adata.obs['Sample']=='Sample_02'],
    shape=None,
    color=[
        "leiden",
    ],
    groups=['7','8'],
    wspace=0.4,
    figsize=(8,8)
)

In [ ]:
sc.pl.pca(adata, color='leiden')

In [ ]:
adata.obs['total_counts'].median()

In [ ]:
adata.X[adata.X!=0]= 1
adata = adata[:,[x for x in adata.var_names if x not in adata.var.sort_values('total_counts',ascending=False).head(5).index.tolist()]]
ct_df = pd.DataFrame(index=adata.obs_names, columns=[f'{ct}' for ct in ct_marker_dict.keys()])
for ct,genes in ct_marker_dict.items():
    name = f'{ct} markers expressed'
    adata.obs[name] = adata[:,[x for x in genes if x in adata.var_names]].X.todense().sum(axis=1)
    ct_df[ct] = adata.obs[name].copy()
adata.obs['Cell Subtype_ind'] = ct_df.idxmax(axis=1)
sc.pl.umap(adata,color='Cell Subtype_ind')

In [ ]:
sample_1_leiden_dictionary = {
    '0': 'Uterine smooth muscle',
    '1':'Uterine smooth muscle',
    '2':'Uterine smooth muscle',
    '3':'Matrix Fibroblast',
    '4':'Vascular smooth muscle',
    '5':'Uterine smooth muscle',
    '6':'Macrovascular',
    '7':'Uterine smooth muscle',
    '8':'Myeloid',
    '9':'Uterine smooth muscle',
    '10':'Uterine smooth muscle',
    '11':'Uterine smooth muscle',
    '12':'Myeloid',
    '13':'Uterine smooth muscle',}
adata.obs['Cell Subtype']= [sample_1_leiden_dictionary[x] for x in adata.obs['leiden']]
sc.pl.umap(adata, color='Cell Subtype')

In [ ]:
ser_counts = adata.obs["leiden"].value_counts()
ser_counts.name = "cell counts"
meta_leiden = pd.DataFrame(ser_counts)

cat_name = "leiden"
sig_leiden = pd.DataFrame(
    columns=adata.var_names, index=adata.obs[cat_name].cat.categories
)
for clust in adata.obs[cat_name].cat.categories:
    sig_leiden.loc[clust] = adata[adata.obs[cat_name].isin([clust]), :].X.mean(0)
sig_leiden = sig_leiden.transpose()
leiden_clusters = ["Leiden-" + str(x) for x in sig_leiden.columns.tolist()]
sig_leiden.columns = leiden_clusters
meta_leiden.index = sig_leiden.columns.tolist()
meta_leiden["leiden"] = pd.Series(
    meta_leiden.index.tolist(), index=meta_leiden.index.tolist()
)

In [ ]:
meta_leiden

In [ ]:
adata.var_names

In [ ]:
common_marker_genes = gene_df.loc[gene_df['Reason'].str.contains('ct_marker')]
common_marker_genes = common_marker_genes.loc[~common_marker_genes['Reason'].str.contains(',')]
common_marker_genes['Reason'] = common_marker_genes['Reason'].str.split(':').str[1]
common_marker_genes['Reason'] = [x[:-2] for x in common_marker_genes['Reason']]
common_marker_genes['Function'] = common_marker_genes['Reason']
common_marker_genes = common_marker_genes.loc[[x for x in common_marker_genes.index if x in adata.var_names.tolist()]]
common_marker_genes

In [ ]:
meta_gene = pd.DataFrame(index=sig_leiden.index.tolist())
meta_gene["info"] = pd.Series("", index=meta_gene.index.tolist())
meta_gene["Markers"] = pd.Series("N.A.", index=sig_leiden.index.tolist())
meta_gene.loc[common_marker_genes.index, "Markers"] = common_marker_genes['Function']
meta_leiden["Cell_Type"] = pd.Series("N.A.", index=meta_leiden.index.tolist())
num_top_genes = 30
for inst_cluster in sig_leiden.columns.tolist():
    top_genes = (
        sig_leiden[inst_cluster]
        .sort_values(ascending=False)
        .index.tolist()[:num_top_genes]
    )

    inst_ser = meta_gene.loc[top_genes, "Markers"]
    inst_ser = inst_ser[inst_ser != "N.A."]
    ser_counts = inst_ser.value_counts()

    max_count = ser_counts.max()

    max_cat = "_".join(sorted(ser_counts[ser_counts == max_count].index.tolist()))
    max_cat = max_cat.replace(" marker", "").replace(" ", "-")

    print(inst_cluster, max_cat)
    meta_leiden.loc[inst_cluster, "Cell_Type"] = max_cat

# rename clusters
meta_leiden["name"] = meta_leiden.apply(
    lambda x: str(x["Cell_Type"]) + "_" + str(x["leiden"]), axis=1
)
leiden_names = meta_leiden["name"].values.tolist()
meta_leiden.index = leiden_names


# transfer cell type labels to single cells
leiden_to_cell_type = meta_leiden.copy()
leiden_to_cell_type.set_index("leiden", inplace=True)
leiden_to_cell_type.index.name = None

adata.obs["Cell_Type"] = adata.obs["leiden"].apply(
    lambda x: leiden_to_cell_type.loc["Leiden-" + str(x), "Cell_Type"]
)
adata.obs["Cluster"] = adata.obs["leiden"].apply(
    lambda x: leiden_to_cell_type.loc["Leiden-" + str(x), "name"]
)

In [ ]:
meta_gene['Markers'].unique()

In [ ]:
sc.pl.umap(adata,color='Cell_Type')

In [ ]:
sq.gr.interaction_matrix(adata,cluster_key='Cell Subtype',normalized=True)
sq.pl.interaction_matrix(adata,cluster_key='Cell Subtype')

In [ ]:
markers_expressed_table = adata.obs[[f'{ct} markers expressed' for ct in ct_marker_dict.keys()]]
markers_expressed_table
adata.obs['ct_marker_max'] = markers_expressed_table.idxmax(axis=1)
sc.pl.umap(adata, color ='ct_marker_max')

In [ ]:
markers_expressed_table.idxmax(axis=1)

In [ ]:
sq.pl.spatial_scatter(
    adata[adata.obs['Sample']=='Sample_01'],
    shape=None,
    color=[
        "leiden",
    ],
    # groups=['Macrovascular','Vascular smooth muscle'],
    wspace=0.4,
    figsize=(8,8)
)

In [ ]:
sq.pl.spatial_scatter(
    adata,
    shape=None,
    color=[
        "ADGRL3",
    ],
    # groups=['Macrovascular','Vascular smooth muscle'],
    wspace=0.4,
    figsize=(8,8)
)

In [ ]:
sq.pl.spatial_scatter(
    adata2,
    shape='hex',
    color=[
        "Cell Subtype",
    ],
    # groups=['Macrovascular','Vascular smooth muscle'],
    wspace=0.4,
    figsize=(8,8)
)


In [ ]:
sq.pl.spatial_scatter(
    adata2,
    shape=None,
    color=[
        "Cell Subtype",
    ],
    # groups=['Macrovascular','Vascular smooth muscle'],
    wspace=0.4,
    figsize=(8,8)
)

In [ ]:
sq.pl.spatial_scatter(
    adata,
    shape=None,
    color=[
        "DPP6",
    ],
    # groups=['Macrovascular','Vascular smooth muscle'],
    wspace=0.4,
    figsize=(8,8)
)

In [ ]:
sc.pl.embedding(adata,basis='spatial',color='Cell Subtype')

In [ ]:
adata.X[adata.X!=0]= 1
for ct,genes in ct_marker_dict.items():
    name = f'{ct} markers expressed'
    adata.obs[name] = adata[:,[x for x in genes if x in adata.var_names]].X.todense().sum(axis=1)
    sc.pl.umap(adata,color=name)


In [ ]:
sq.pl.spatial_scatter(
    adata, shape=None, color="DPP6", size=1
)

In [ ]:
sq.pl.spatial_scatter(
    adata, shape=None, color="NOTCH3", size=1
)

### Spatial stuff

In [ ]:
sq.gr.spatial_neighbors(adata, coord_type="generic", delaunay=True)


In [ ]:
sq.gr.centrality_scores(adata, cluster_key="leiden")


In [ ]:
sq.pl.centrality_scores(adata, cluster_key="leiden", figsize=(16, 5))


In [ ]:
sq.gr.co_occurrence(adata, cluster_key="leiden")


In [ ]:
sq.pl.co_occurrence(adata,cluster_key='leiden')